In [ ]:
import joblib

In [ ]:
model = joblib.load('models/best_model.pkl')

In [ ]:
# FAST API

"""
main.py
$ pip install "fastapi[all]"
$ uvicorn main:app --reload
http://127.0.0.1:8000/redoc

POST requests in the bash console:

curl \
  --header "Content-Type: application/json" \
  --request POST \
  --data '{"x1": "1.25", "x2": "1"}' \
  http://localhost:8000/predict

or like this:

curl -X 'POST' \
  'http://127.0.0.1:8000/predict' \
  -H 'accept: aplication/json' \
  -H 'Content-Type: application/json' \
  -d '{
    "x1": 1.75,
    "x2": 0,
    "x3": "small"
    }'

FastAPI tutorial:
https://fastapi.tiangolo.com/tutorial/
"""

import uvicorn  # not needed if  $ uvicorn main:app
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
from sklearn.linear_model import LinearRegression
import joblib


# Load your model
model = joblib.load('models/best_model.pkl')


app = FastAPI()


class Data(BaseModel):
    x1: float
    x2: int
    x3: str | None = None


@app.post("/predict")
async def predict(data: Data):
    d = data.dict()
    ypred, = model.predict([[d['x1'], d['x2']]])
    return {'prediction': ypred}


@app.get("/predict", response_class=HTMLResponse)
async def predict(x1: float, x2: float, x3: str = None):
    """http://127.0.0.1:8000/predict?x1=-7.25&x2=1"""
    ypred, = model.predict([[x1, x2]])
    return HTMLResponse(f"Our prediction: {ypred}")


@app.get("/")
async def root():
    return HTMLResponse("Hello World")


@app.get("/items/{item_id}")
def get_item(item_id: int):
    return {'item_id': item_id}


@app.get("/users/{user_id}/items/{item_id}")
async def get_user_item(user_id: int,
                         item_id: str,
                         q: str | None = None):
    """http://127.0.0.1:8000/users/123/items/bookshelf?q=title:%22The%20Book%22%20AND%20author:%22M.Wright%22"""
    return {"user_id": user_id, "item_id": item_id, "q": q}


if __name__ == '__main__':
    uvicorn.run(app, host='127.0.0.1', port=8000)
# or bash: $ uvicorn main:app --reload
